# 🚀 LangGraph Studio 1.0 (Alpha) on Google Colab

This notebook sets up **LangGraph Studio v1.0-alpha** with tunneling to run in Google Colab.

## 📋 Prerequisites

You'll need:
1. **LangSmith API Key** - Get from [smith.langchain.com/settings](https://smith.langchain.com/settings) (free signup)
2. **OpenAI API Key** - Get from [platform.openai.com](https://platform.openai.com)

## ⚠️ Note
LangGraph 1.0 is currently in **alpha** - APIs may change before stable release.

---

## Step 1: Install LangGraph 1.0 Alpha

In [ ]:
print("📦 Installing LangGraph 1.0 (alpha) and dependencies...")
!pip install -q --pre "langgraph>=1.0.0a0" "langgraph-cli>=1.0.0a0" langchain-openai langchain-core

print("\n✅ Installation complete!")
print("\nℹ️  LangGraph 1.0 is currently in alpha. API may change.")

## Step 2: Set Up API Keys

### Option A: Using Colab Secrets (Recommended)
1. Click the 🔑 icon in the left sidebar
2. Add `LANGSMITH_API_KEY`
3. Add `OPENAI_API_KEY`
4. Enable notebook access for both

### Option B: Manual Entry (Run the cell below and uncomment)

In [ ]:
import os
from google.colab import userdata

# Option A: Try to load from Colab secrets
try:
    os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ API keys loaded from Colab secrets")
except:
    print("⚠️  Could not load from secrets. Using manual entry...")
    
    # Option B: Manual entry (uncomment and add your keys)
    # os.environ['LANGSMITH_API_KEY'] = 'lsv2_pt_...'  # Your LangSmith API key
    # os.environ['OPENAI_API_KEY'] = 'sk-...'  # Your OpenAI API key
    # print("✅ API keys set manually")
    
    print("\n❌ Please set your API keys using Option A or B above!")

# Set additional environment variables
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'langgraph-v1-colab'

print("\n📊 LangSmith tracing enabled")
print(f"📁 Project: {os.environ.get('LANGCHAIN_PROJECT')}")

## Step 3: Create Project Structure

In [ ]:
import os

# Create project directory
project_dir = '/content/langgraph_app'
os.makedirs(project_dir, exist_ok=True)

print(f"✅ Project directory created: {project_dir}")

## Step 4: Create Agent Code (v1.0 API)

In [ ]:
agent_code = '''from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode, tools_condition

def create_agent():
    """Create a simple LangGraph v1.0 agent with tools"""
    
    # Define simple tools
    def get_weather(location: str) -> str:
        """Get the weather for a location.
        
        Args:
            location: The city or location to get weather for
        """
        # Mock weather data
        weather_data = {
            "san francisco": "Foggy and 55°F",
            "new york": "Sunny and 68°F",
            "london": "Rainy and 52°F",
            "tokyo": "Clear and 72°F"
        }
        location_lower = location.lower()
        return weather_data.get(location_lower, f"The weather in {location} is sunny and 70°F")
    
    def multiply(a: float, b: float) -> float:
        """Multiply two numbers.
        
        Args:
            a: First number
            b: Second number
        """
        return a * b
    
    def search_web(query: str) -> str:
        """Search the web for information.
        
        Args:
            query: The search query
        """
        return f"Here are some results for '{query}': [Mock search results]"       
    
    tools = [get_weather, multiply, search_web]
    
    # Create LLM with tools
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    llm_with_tools = llm.bind_tools(tools)
    
    # Define the agent node
    def call_model(state: MessagesState):
        response = llm_with_tools.invoke(state["messages"])
        return {"messages": [response]}
    
    # Build the graph using v1.0 API
    builder = StateGraph(MessagesState)
    
    # Add nodes
    builder.add_node("agent", call_model)
    builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    builder.add_edge(START, "agent")
    builder.add_conditional_edges("agent", tools_condition)
    builder.add_edge("tools", "agent")
    
    # Compile and return the graph
    return builder.compile()
'''

# Write agent code to file
with open(f'{project_dir}/agent.py', 'w') as f:
    f.write(agent_code)

print("✅ Agent code created at /content/langgraph_app/agent.py")
print("\n🤖 Agent includes tools for:")
print("   - Weather lookup")
print("   - Mathematical operations")
print("   - Web search (mock)")

## Step 5: Create Configuration Files

In [ ]:
import json

# Create langgraph.json configuration
config = {
    "dependencies": ["."],
    "graphs": {
        "agent": "./agent.py:create_agent"
    },
    "env": ".env"
}

with open(f'{project_dir}/langgraph.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✅ langgraph.json created")

# Create .env file
env_content = f"""LANGSMITH_API_KEY={os.environ.get('LANGSMITH_API_KEY', '')}
OPENAI_API_KEY={os.environ.get('OPENAI_API_KEY', '')}
LANGCHAIN_TRACING_V2=true
LANGCHAIN_PROJECT=langgraph-v1-colab
"""

with open(f'{project_dir}/.env', 'w') as f:
    f.write(env_content)

print("✅ .env file created")
print("\n📁 Project structure:")
print("   langgraph_app/")
print("   ├── agent.py")
print("   ├── langgraph.json")
print("   └── .env")

## Step 6: Verify Setup

In [ ]:
import os

print("🔍 Verifying setup...\n")

# Check files
files_to_check = ['agent.py', 'langgraph.json', '.env']
all_files_exist = True

for file in files_to_check:
    file_path = f'{project_dir}/{file}'
    if os.path.exists(file_path):
        print(f"✅ {file} exists")
    else:
        print(f"❌ {file} missing")
        all_files_exist = False

# Check API keys
print("\n🔑 Checking API keys...")
if os.environ.get('LANGSMITH_API_KEY'):
    print("✅ LANGSMITH_API_KEY is set")
else:
    print("❌ LANGSMITH_API_KEY is missing")
    all_files_exist = False

if os.environ.get('OPENAI_API_KEY'):
    print("✅ OPENAI_API_KEY is set")
else:
    print("❌ OPENAI_API_KEY is missing")
    all_files_exist = False

if all_files_exist:
    print("\n" + "="*60)
    print("✨ Setup complete! Ready to start LangGraph Studio")
    print("="*60)
else:
    print("\n" + "="*60)
    print("⚠️  Setup incomplete. Please fix the issues above.")
    print("="*60)

## Step 7: Start LangGraph Studio with Tunnel

⚠️ **Important Notes:**
- This cell will run indefinitely until you stop it
- The tunnel URL will be displayed in the output
- Click the URL to access Studio UI
- To stop the server: Click the ⏹️ stop button or press Ctrl+C
- Colab may timeout after ~12 hours of inactivity

In [ ]:
import os

# Change to project directory
os.chdir(project_dir)

print("🚀 Starting LangGraph Studio with tunnel...")
print("\n" + "="*60)
print("⏳ This may take a minute to start up...")
print("📺 Look for the tunnel URL in the output below")
print("🔗 Click the URL to access Studio UI")
print("="*60 + "\n")

# Start the server with tunnel
!langgraph dev --tunnel

## 🎉 Success!

If everything worked, you should see:
1. A tunnel URL (like `https://xxxxx.loca.lt`)
2. A Studio UI URL

### What you can do in Studio:
- 💬 Chat with your agent
- 🔍 See the graph visualization
- 🐛 Debug tool calls and agent decisions
- 📊 View traces in LangSmith

### Example prompts to try:
- "What's the weather in San Francisco?"
- "Calculate 123 times 456"
- "Search for information about LangGraph"

---

## 📚 Resources
- [LangGraph v1 Docs](https://docs.langchain.com/oss/python/langgraph/studio)
- [LangSmith](https://smith.langchain.com)
- [LangGraph GitHub](https://github.com/langchain-ai/langgraph)

## ⚠️ Troubleshooting

**If the server won't start:**
- Check that API keys are set correctly
- Verify all files were created in Step 6
- Try restarting the runtime (Runtime → Restart runtime)

**If you can't access the tunnel URL:**
- Some networks block tunnel services
- Try from a different network or device
- Check if the URL has expired (tunnels timeout)

**If you get import errors:**
- Re-run Step 1 to reinstall packages
- Make sure you're using Python 3.10+